In [ ]:
"""
@Authors:                   @Email:
* Shaiful                     shaiful019@gmail.com
* Rezwanul                    rezwanul.cse@gmail.com
* Oli
* Habibul                     shovon2111@gmail.com
* Tahmid
"""

# Problem: Return to Origin (RTO) Customers Prediction
 Problem class: Supervised Classification
 
 Problem dataset link: Real dataset so no online link.
 
 Problem description: 
 
     RTO (Return to Origin) is a Classification Problem in eCommerce platforms where Many customers cancel their order when the   product is already on shipping. Then customer don't response and the product is return back to office.
 
 Problem Task:
     Have to predict who will Cancel the product order after product is already on shipping.

# Importing libraries

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

# Feature or column description

COLUMN DESCRIPTIONS:
* Order Id
* User Id
* Order Value (In BDT, amount of the order)
* Payment Type (How, customers paid for it. MPD/MPC = Manual, means Cash on delivery. MPS = Bkash, Rocket type mobile payments. * OPS = online credit card, debit card. ADC = AD Cash (can be earned, can be recharged), EMI = Monthly EMI Purchase )
* District (Where customers are getting delivered)
* OrderSource (Android, Mobile Site or Desktop Site)
* Order Type for Delivery ( Express = Quick delivery and paid more for it, Normal = average delivery)
* Delivery Charge ( for the delivery)
* Order Placing Date (When customer placed the order)
* Order Confirmation date (Office make calls to confirm the order. It call is made late, they might not want the product later)
* IsCartOrder (is it a cart order or single product order)
* OrderFromPromotionalEvent (Is product purchased in discount from a promotional event or not)
* courierService (Courier service used by the customer)
* isRTO ( Order canceled after product is already on shipping )

# Load Dataset & Merging the datasets

In [25]:
df_1 = pd.read_csv("dataset/RTOdata.csv")
df_2 = pd.read_csv("dataset/RTO_data_new.csv")

# Merging
df = pd.concat([df_1, df_2], ignore_index=True)
# type(df_1)
# type(df_2)
df.shape

(20000, 14)

In [26]:
# df.head()
df.tail()

,OrderId,UserId,OrderValue,PaymentType,District,OrderSource,OrderType,DeliveryCharge,OrderPlacedDay,OrderConfirmDayOverPhone,IsCartOrder,OrderFromPromotionalEvent,courierService,isRTO
19995,2610588,901210,187,MPD,Nilphamari,android,normal,60.0,7/17/2019 19:29,7/17/2019 13:33,No,Yes,Bangladesh Postal Service ...,Yes
19996,2610586,528636,235,MPS,Sylhet City,android,normal,0.0,7/17/2019 19:29,7/18/2019 5:55,No,Yes,E-Desh ...,No
19997,2610583,649604,170,MPS,Bagerhat,android,normal,25.0,7/17/2019 19:27,7/17/2019 15:42,No,Yes,Paperfly ...,No
19998,2610576,967577,409,MPD,Dhaka,android,normal,25.0,7/17/2019 19:20,7/17/2019 13:28,No,No,E-Desh ...,No
19999,2610572,970326,402,MPD,Moulvibazar,android,normal,60.0,7/17/2019 19:18,7/17/2019 13:45,No,Yes,Paperfly ...,Yes


# Data Exploration

## veriable Identification

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 14 columns):
OrderId                      20000 non-null int64
UserId                       20000 non-null int64
OrderValue                   20000 non-null int64
PaymentType                  20000 non-null object
District                     20000 non-null object
OrderSource                  20000 non-null object
OrderType                    19319 non-null object
DeliveryCharge               19999 non-null float64
OrderPlacedDay               20000 non-null object
OrderConfirmDayOverPhone     20000 non-null object
IsCartOrder                  20000 non-null object
OrderFromPromotionalEvent    20000 non-null object
 courierService              10000 non-null object
isRTO                        20000 non-null object
dtypes: float64(1), int64(3), object(10)
memory usage: 2.1+ MB


In [28]:
df.dtypes

OrderId                        int64
UserId                         int64
OrderValue                     int64
PaymentType                   object
District                      object
OrderSource                   object
OrderType                     object
DeliveryCharge               float64
OrderPlacedDay                object
OrderConfirmDayOverPhone      object
IsCartOrder                   object
OrderFromPromotionalEvent     object
 courierService               object
isRTO                         object
dtype: object

### Drop Unnecessary Columns

In [29]:
df.drop("OrderId", axis=1, inplace=True)
df.tail()

,UserId,OrderValue,PaymentType,District,OrderSource,OrderType,DeliveryCharge,OrderPlacedDay,OrderConfirmDayOverPhone,IsCartOrder,OrderFromPromotionalEvent,courierService,isRTO
19995,901210,187,MPD,Nilphamari,android,normal,60.0,7/17/2019 19:29,7/17/2019 13:33,No,Yes,Bangladesh Postal Service ...,Yes
19996,528636,235,MPS,Sylhet City,android,normal,0.0,7/17/2019 19:29,7/18/2019 5:55,No,Yes,E-Desh ...,No
19997,649604,170,MPS,Bagerhat,android,normal,25.0,7/17/2019 19:27,7/17/2019 15:42,No,Yes,Paperfly ...,No
19998,967577,409,MPD,Dhaka,android,normal,25.0,7/17/2019 19:20,7/17/2019 13:28,No,No,E-Desh ...,No
19999,970326,402,MPD,Moulvibazar,android,normal,60.0,7/17/2019 19:18,7/17/2019 13:45,No,Yes,Paperfly ...,Yes


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
UserId                       20000 non-null int64
OrderValue                   20000 non-null int64
PaymentType                  20000 non-null object
District                     20000 non-null object
OrderSource                  20000 non-null object
OrderType                    19319 non-null object
DeliveryCharge               19999 non-null float64
OrderPlacedDay               20000 non-null object
OrderConfirmDayOverPhone     20000 non-null object
IsCartOrder                  20000 non-null object
OrderFromPromotionalEvent    20000 non-null object
 courierService              10000 non-null object
isRTO                        20000 non-null object
dtypes: float64(1), int64(2), object(10)
memory usage: 2.0+ MB


## Univariate Analysis

In [31]:
df.describe()

,UserId,OrderValue,DeliveryCharge
count,20000.000000,20000.000000,19999.000000
mean,764778.585550,323.887900,32.538977
std,250551.644033,368.370536,24.673762
min,83.000000,0.000000,0.000000
25%,615149.000000,165.000000,25.000000
50%,887995.000000,235.000000,25.000000
75%,954996.000000,366.250000,55.000000
max,986270.000000,12800.000000,500.000000


## Missing Value Analysis

In [32]:
## Missing value
df.isna().sum()

UserId                           0
OrderValue                       0
PaymentType                      0
District                         0
OrderSource                      0
OrderType                      681
DeliveryCharge                   1
OrderPlacedDay                   0
OrderConfirmDayOverPhone         0
IsCartOrder                      0
OrderFromPromotionalEvent        0
 courierService              10000
isRTO                            0
dtype: int64

#### Here we see the OrderType, DeliveryCharge and courierService has missing value. 

Replacing empty/null values with "NaN"

In [33]:
df=df.fillna("NaN")

In [34]:
df.isna().sum()

UserId                       0
OrderValue                   0
PaymentType                  0
District                     0
OrderSource                  0
OrderType                    0
DeliveryCharge               0
OrderPlacedDay               0
OrderConfirmDayOverPhone     0
IsCartOrder                  0
OrderFromPromotionalEvent    0
 courierService              0
isRTO                        0
dtype: int64

## Class Imbalance Analysis

In [35]:
for col in df.columns:
    print(df[col].value_counts())
    print()
    print("-----------------------------------------------------------")
    print()

816086    55
928321    42
893963    32
807767    29
953939    28
945288    23
616639    21
957998    20
869083    20
787154    20
973042    19
975437    17
978426    17
893567    17
955997    17
554254    16
97369     15
867826    14
630761    14
438153    14
581564    14
978900    14
954959    14
949513    14
927521    13
974400    13
737439    13
824313    13
809875    13
903541    13
          ..
983913     1
799519     1
977700     1
979751     1
854840     1
961340     1
158532     1
957258     1
955211     1
984313     1
975701     1
979799     1
959325     1
949094     1
926573     1
556920     1
930671     1
975733     1
959357     1
932734     1
865151     1
777090     1
938883     1
983542     1
645510     1
955275     1
897935     1
979863     1
878245     1
954370     1
Name: UserId, Length: 10932, dtype: int64

-----------------------------------------------------------

0       830
250     497
120     450
350     373
185     354
199     341
499     330
140     304
290    

Here we see the OrderSource field has Imbalance data as "-1" which didn't mean anything to us. As it can't be determine as order source we're going to marge this data with "Others" value. 

In [36]:
df["OrderSource"] = df["OrderSource"].replace(to_replace ="-1", value ="Others")

In [37]:
df

,UserId,OrderValue,PaymentType,District,OrderSource,OrderType,DeliveryCharge,OrderPlacedDay,OrderConfirmDayOverPhone,IsCartOrder,OrderFromPromotionalEvent,courierService,isRTO
0,663791,168,MPD,Dhaka,android,normal,25,6/26/2019 9:20,6/26/2019 3:30,No,Yes,NaN,No
1,961083,240,MPD,Dhaka,mobile-site,normal,35,6/25/2019 19:05,6/25/2019 13:40,No,Yes,NaN,No
2,848517,120,MPD,Dhaka,desktop,normal,35,6/25/2019 15:28,6/25/2019 9:27,No,Yes,NaN,Yes
3,276304,185,MPD,Dhaka,android,normal,25,6/25/2019 13:27,6/25/2019 7:27,Yes,No,NaN,No
4,955384,201,MPD,Dhaka,android,normal,25,6/25/2019 12:13,6/25/2019 6:56,No,Yes,NaN,No
5,832391,155,MPD,Dhaka,android,normal,25,6/25/2019 11:07,6/25/2019 13:11,No,Yes,NaN,No
6,832391,105,MPD,Dhaka,android,normal,25,6/25/2019 10:50,6/25/2019 13:11,No,No,NaN,No
7,442723,120,MPD,Dhaka,android,normal,25,6/25/2019 10:50,6/25/2019 4:49,No,Yes,NaN,No
8,959412,499,MPD,Cox's-Bazar,android,normal,55,6/25/2019 10:40,6/25/2019 5:05,No,No,NaN,Yes
9,959412,159,MPD,Cox's-Bazar,android,normal,55,6/25/2019 10:30,6/25/2019 4:36,No,No,NaN,Yes
